In [1]:
import pandas as pd  
from sklearn.model_selection import train_test_split
import xgboost as xgb
import os
import math

from xai4chem import MorganFingerprint, RDKitDescriptor, DatamolDescriptor, AccFgFingerprint
from xai4chem import Explainer
from xai4chem.supervised import Regressor, Classifier

#data = pd.read_csv("../data/plasmodium_falciparum_3d7_ic50.csv") #data path
data = pd.read_csv("/home/jason/Downloads/h3d_solubility_74_July2025_preds.csv")

2025-12-19 13:14:14.570230: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Imported lazytransform v1.18. 

Imported featurewiz 0.6.0. Use the following syntax:
    >>> wiz = FeatureWiz(feature_engg = '', nrows=None, transform_target=True,
        		category_encoders="auto", auto_encoders='VAE', ae_options={},
        		add_missing=False, imbalanced=False, verbose=0)
    >>> X_train_selected, y_train = wiz.fit_transform(X_train, y_train)
    >>> X_test_selected = wiz.transform(X_test)
    >>> selected_features = wiz.features
    


In [2]:
# Extract SMILES and target values
smiles = data["smiles"]
target = data["activity"] #target value's column_name
#log_target = [math.log10(val) for val in data["exp"]]
#data.loc[:, "log_exp"] = log_target
#target = data["log_exp"]

# Split data into training and test sets
smiles_train, smiles_valid, y_train, y_valid = train_test_split(smiles, target, test_size=0.2, random_state=42)

# Reset indices
smiles_train.reset_index(drop=True, inplace=True)
smiles_valid.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_valid.reset_index(drop=True, inplace=True)

In [3]:
#descriptor = MorganFingerprint()
descriptor = DatamolDescriptor()
#descriptor = AccFgFingerprint()

# Transform the data
smiles_train_transformed = descriptor.fit(smiles_train)
smiles_valid_transformed = descriptor.transform(smiles_valid)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1145/1145 [00:03<00:00, 372.84it/s]


In [4]:
# Specify the output folder where evaluation metrics and interpretability plots will be saved.
output_folder = "example_folder_path"
explainer_path = os.path.join(output_folder, "regressor.joblib")

# Specify fingerprints to use: 'morgan' / 'accfg' / 'datamol' for Morgan Bits / Functional Groups / Global Properties respectively
regressor = Regressor(output_folder, k=2048, fingerprints='datamol') 

In [5]:
# Train the model
regressor.fit(smiles_train_transformed, y_train)

# Evaluate the model
regressor.evaluate(smiles_valid_transformed, smiles_valid, y_valid)

[I 2025-12-19 13:14:32,100] A new study created in memory with name: no-name-be2ddb73-e8bd-48b1-ba89-b0a327142fb2


Number of input features is less than or equal to 2048. Using all features.


[I 2025-12-19 13:14:33,337] Trial 0 finished with value: 0.18971479104512046 and parameters: {'lambda': 3, 'alpha': 1, 'gamma': 6, 'learning_rate': 0.36121472053804704, 'colsample_bytree': 0.8, 'colsample_bynode': 0.7, 'n_estimators': 60, 'min_child_weight': 66, 'max_depth': 16, 'subsample': 0.8, 'random_state': 42}. Best is trial 0 with value: 0.18971479104512046.
[I 2025-12-19 13:14:33,394] Trial 1 finished with value: 0.2149912172476271 and parameters: {'lambda': 2, 'alpha': 2, 'gamma': 14, 'learning_rate': 0.08124935117121225, 'colsample_bytree': 1.0, 'colsample_bynode': 0.7, 'n_estimators': 188, 'min_child_weight': 43, 'max_depth': 13, 'subsample': 0.4, 'random_state': 0}. Best is trial 0 with value: 0.18971479104512046.
[I 2025-12-19 13:14:33,455] Trial 2 finished with value: 0.21498891518089908 and parameters: {'lambda': 4, 'alpha': 0, 'gamma': 14, 'learning_rate': 0.28062560908563255, 'colsample_bytree': 0.6, 'colsample_bynode': 1.0, 'n_estimators': 159, 'min_child_weight': 35,

Best parameters for XGBoost: {'lambda': 0, 'alpha': 2, 'gamma': 0, 'learning_rate': 0.6155031782666202, 'colsample_bytree': 1.0, 'colsample_bynode': 0.6, 'n_estimators': 187, 'min_child_weight': 3, 'max_depth': 14, 'subsample': 1.0, 'random_state': 0}


{'Mean Squared Error': 0.0629,
 'Root Mean Squared Error': 0.2509,
 'Mean Absolute Error': 0.2171,
 'R-squared Score': -0.324,
 'Explained Variance Score': -0.0578}

In [6]:
# Produce the interpretability plots for the overall model features 
regressor.explain(smiles_train)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4579/4579 [00:12<00:00, 375.58it/s]


Explaining model


In [7]:
# You can save the trained model
regressor.save_model(explainer_path) #pass the filename

In [8]:
regressor.load_model(explainer_path)

In [ ]:
# Produce interpretability plots for individual molecules
regressor.explain_mol_atoms("CC(=O)Nc1ccc(O)cc1", atomInfo=False)